# Investigation of Deep learning architecture using PyTorch.
Most likely as it pertains to image classification.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

%cd /content/drive/MyDrive/Portfolio/Deep Learning With PyTorch/

/content/drive/MyDrive/Portfolio/Deep Learning With PyTorch


## Brushing up on PyTorch library


### Tensors
A simple scalar (e.g., 1) can be represented as a tensor of rank 0, a vector is rank 1, an n × n matrix is rank 2, and so on. In the previous
example, we created a rank 2 tensor with random values by using torch.rand().

In [ ]:
#Tensor rank 2
print(torch.rand(2,2))

#Can also be created from lists
x = torch.tensor([[0,0,1],[1,1,1],[0,0,0]])
print(x)

#Can change an element using python standard indexing
x[2][-1] = 5 #Can also use negative indices to move from the end inwards
print(x)

#Useful creation functions
print(torch.zeros(2,2))
print(torch.ones(2,2))

tensor([[0.9289, 0.2404],
        [0.9587, 0.5066]])
tensor([[0, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])
tensor([[0, 0, 1],
        [1, 1, 1],
        [0, 0, 5]])
tensor([[0., 0.],
        [0., 0.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[2., 2.]])


In [ ]:
#Tensor addition
print(torch.ones(1,2) + torch.ones(1,2))

#Extracting max value
x = torch.rand(2,2)
print(x)

# Getting the max from the tensor and extracting the value
print(x.max())
print(x.max().item())
#Getting the index of the max from the tensor and extracting the value (Useful for NN class estimation determination)
print(x.argmax())
print(x.argmax().item())

tensor([[2., 2.]])
tensor([[0.9343, 0.6951],
        [0.3739, 0.1860]])
tensor(0.9343)
0.934316098690033
tensor(0)
0


PyTorch's tensor operations often return a new tensor as a result but some will have an "in-place" update version that has an underscore after the method name.

In [ ]:
x = torch.rand(2,2)
print(x)
x.log2() #will return a tensor that I'm currently not assigning
#The original tensor will remain unchanged
print(x)
#In-place
x.log2_()
#Original tensor will be changed
print(x)

tensor([[0.4778, 0.3666],
        [0.2653, 0.9942]])
tensor([[0.4778, 0.3666],
        [0.2653, 0.9942]])
tensor([[-1.0654, -1.4476],
        [-1.9144, -0.0084]])


Other functionality to explore as it comes up:
view
reshapeing

# Image classification with NN
Cats & Fish
Using ImageNet dataset

## Data Loaders

Normalizing is important because a lot of multiplication will be happening as the
input passes through the layers of the neural network; keeping the incoming values
between 0 and 1 prevents the values from getting too large during the training phase
(known as the exploding gradient problem). And that magic incarnation is just the
mean and standard deviation of the ImageNet dataset as a whole. You could calculate
it specifically for this fish and cat subset, but these values are decent enough. (If you
were working on a completely different dataset, you’d have to calculate that mean and
deviation, although many people just use these ImageNet constants and report
acceptable results.)


In [ ]:
import torchvision
from torchvision import transforms

train_data_path = "cat_fish_dataset/train/"
val_data_path = "cat_fish_dataset/val/"
test_data_path = "cat_fish_dataset/test/"

transforms = transforms.Compose([
    transforms.Resize(64), #resolution 64 x 64
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms)

In [ ]:
train_data.samples

[('cat_fish_dataset/train/cat/052cat.jpg', 0),
 ('cat_fish_dataset/train/cat/070321-toygers-photos_big.jpg', 0),
 ('cat_fish_dataset/train/cat/10002817_5944408634.jpg', 0),
 ('cat_fish_dataset/train/cat/1004525_cba96ba3c3.jpg', 0),
 ('cat_fish_dataset/train/cat/1004527_527c0c32ce.jpg', 0),
 ('cat_fish_dataset/train/cat/1004528_a111209743.jpg', 0),
 ('cat_fish_dataset/train/cat/100739452_5be2c11557.jpg', 0),
 ('cat_fish_dataset/train/cat/1010205192_6ce1a37dc5.jpg', 0),
 ('cat_fish_dataset/train/cat/1010884970_1aa4af4701.jpg', 0),
 ('cat_fish_dataset/train/cat/1011749126_44a195db4c.jpg', 0),
 ('cat_fish_dataset/train/cat/1011749488_b465af926a.jpg', 0),
 ('cat_fish_dataset/train/cat/1011749668_fc48f28767.jpg', 0),
 ('cat_fish_dataset/train/cat/1015160097_c4103501de.jpg', 0),
 ('cat_fish_dataset/train/cat/1020320535_13679b940e.jpg', 0),
 ('cat_fish_dataset/train/cat/1029412358_7ee17550fc.jpg', 0),
 ('cat_fish_dataset/train/cat/10298902.jpg', 0),
 ('cat_fish_dataset/train/cat/1030251986_431

Work continued locally on GREED...